In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import re
import camelot
import tabula
import warnings
import codecs
import math

warnings.simplefilter('ignore', UserWarning)
pd.set_option('display.max_rows', 100)
codecs.register_error('none', lambda e: ('', e.end))

Duplicate key in file WindowsPath('C:/ProgramData/Anaconda3/lib/site-packages/matplotlib/mpl-data/matplotlibrc'), line 250 ('font.family: IPAexGothic')



# 全銘柄PDFデータ抽出の流れ

各銘柄のバランスシート上のデータ（流動資産など）をPDFから、PDF表抽出ライブラリーにより抽出する。<br>
※バランスシートのデータを記載しているWebページが存在しないため、各企業の決算短信のPDFからバランスシートの表を抽出してスクレイピングするしかない。<br><br>


PDF表抽出ライブラリーは、camelotとtabulaの2種類がある。それぞれ抽出できない銘柄がいくつか存在する。<br>
camelotの方が抽出できる銘柄が多いので、camelotで抽出後、残りをtabulaで抽出する。<br>
手順1：camelotで抽出（取込エラーが出た銘柄は全て③tabulaでの抽出にまわす）<br>
　　　→取込が出来た銘柄一覧の表を作成<br>
　　　→余分な文字が多数含まれているので、値の加工を行う<br>
　　　→取込エラーが出ずにPDFの読み込みには成功したものの、十分なデータが抽出できなかった銘柄に対して、<br>
　　　　<b>削除銘柄</b>(十分にデータが抽出できていない銘柄)と<br>
　　　　<b>再抽出銘柄</b>(取込時のコードを変えればうまく抽出できそうな銘柄)の選別<br>
　　　→削除銘柄と再抽出銘柄を表から削除する（再抽出銘柄は②で再抽出する）<br>
手順2：camelotで再抽出銘柄のデータを再抽出<br>
　　　→取込が出来た銘柄一覧の表を作成<br>
　　　→余分な文字が多数含まれているので、値の加工を行う<br>
　　　→十分にデータが抽出できていない銘柄を表から削除する<br>
手順3：tabulaで抽出<br>
　　　→取込が出来た銘柄一覧の表を作成<br>
　　　→余分な文字が多数含まれているので、値の加工を行う<br>
　　　→十分にデータが抽出できていない銘柄を表から削除する<br>
手順4：以上で抽出した手順1、手順2、手順3のデータを結合して完成<br>
手順5：結合したデータのMySQLへの取込み

In [2]:
#手順1：camelotで抽出
def get_pdf_camelot(start_url, code):
    #time.sleep(1)
    response = requests.get(start_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    if 'リクエストされたページがみつかりませんでした' in soup.text:
        print('東証以外の銘柄のため、PDF見つからず')
        pass
    else:
        table = soup.find('table', attrs={'class': 'Quote'})
        try:
            #訂正があった場合でも、訂正前のPDFを取得する
            for i in range(0, 5):
                if '訂正' in table.find_all('td', text = re.compile('.*決算短信.*'))[i].text\
                or '補足' in table.find_all('td', text = re.compile('.*決算短信.*'))[i].text:
                    pass
                else:
                    target_pdf = table.find_all('td', text = re.compile('.*決算短信.*'))[i].find('a').get('href')
                    break
        #最近上場した企業は、決算短信がまだ存在しない
        except Exception as e:
                print('決算短信が存在しない')
                print(e)
                print('\n')
                return 
        try:
            tables = camelot.read_pdf(target_pdf, flavor='stream', pages = '4-end', table_areas=['0,800,700,0'])
            get_features_camelot(tables, code)

        except Exception as e:
            tabula_dict[code] = e
            print('取込エラー')
            print(e)
            print('\n')
            pass

def get_features_camelot(tables, code):
    feature_list = ['流動資産合計', '現金及び預金', '現金預金', '有形固定資産', '有形固定資産合計', '投資有価証券', 
                '投資その他の資産合計', '流動負債合計', '固定負債合計']
    df_elements = []
    feature_remove_list = []
    count_tables = 0
    for table in tables:
        df_tbl = table.df
        for feature in feature_list:
            if len(df_tbl[df_tbl.iloc[:,0] == feature]) == 1:
                #資産と負債が違うテーブルのため、df_element1の列がずれる場合がある
                #特徴量と値だけの1行2列のデータフレームを作成
                df_element1 = df_tbl[df_tbl.iloc[:,0] == feature].iloc[:, [0,-1]]
                #後のconcatで列がずれないようにするため、値の列の名前が違うため1に統一する
                df_element2 = df_element1.rename(columns={df_element1.columns[1]: 1})
                df_elements.append(df_element2)
                #資産と負債のテーブルが2つある銘柄がある（連結と単独のテーブル）
                #同じ特徴量を2回取得することを避けるため、一度入れた特徴量は削除リストに登録
                feature_remove_list.append(feature)
        #テーブル処理が1つ終わったら、削除リストに登録された特徴量を元のリストから削除する
        if len(feature_remove_list) != 0:
            for feature_remove in feature_remove_list:
                feature_list.remove(feature_remove)
            feature_remove_list = []

    df_all1 = pd.concat(df_elements).T
    df_all2 = pd.DataFrame([list(df_all1.iloc[1,:])], columns=df_all1.iloc[0,:],)  
    df_all2['code'] = code
    dfs_concat_camelot(df_all2)
    
    
def dfs_concat_camelot(df_all2):
    global df_camelot1
    df_camelot1 = pd.concat([df_camelot1, df_all2], axis=0)

In [3]:
#手順1：camelotで取り込む銘柄コードと、空データフレームの作成
codes = pd.read_csv('全銘柄Webデータ（完成）.csv', encoding=("cp932"), header=0).iloc[:, 0]
df_camelot1 = pd.DataFrame()
tabula_dict = {}

In [4]:
#手順1：camelotでPDFから特徴量をDataFrameへ記入　※8時間くらいかかる
'''
・取込エラー
    Invalid dictionary construct: [/'Type', ...　→　camelotではデータ抽出できないのが、tabulaで抽出できる可能性が高い
    only algorithm code 1 and 2 are supported　→　データ抽出できないPDFである可能性が非常に高い
    No objects to concatenate　→　データ抽出できないPDFである可能性が高い
・決算短信が存在しない
    'NoneType' object has no attribute 'find'　→　上場したてで、決算短信のPDFがまだ存在しない
・東証以外の銘柄のため、PDF見つからず
    東証(東証1部・東証2部・JSQ・マザーズ)以外はPDFが取得できない
'''

for i in range(0, len(codes)):
    code = codes[i]
    start_url = 'http://ke.kabupro.jp/code/{}.htm'.format(code)
    print(start_url, 'i=', i)
    get_pdf_camelot(start_url, code)

http://ke.kabupro.jp/code/1301.htm i= 0
http://ke.kabupro.jp/code/1332.htm i= 1
http://ke.kabupro.jp/code/1333.htm i= 2
取込エラー
Invalid dictionary construct: [/'Type', /'Font', /'BaseFont', /b"b'", /"\\x82l\\x82r\\x96\\xbe\\x92\\xa9'", /'Subtype', /'Type0', /'DescendantFonts', [<PDFObjRef:12>], /'Encoding', /'UniJIS-UTF16-H']


http://ke.kabupro.jp/code/1352.htm i= 3
http://ke.kabupro.jp/code/1375.htm i= 4
http://ke.kabupro.jp/code/1376.htm i= 5
http://ke.kabupro.jp/code/1377.htm i= 6
http://ke.kabupro.jp/code/1379.htm i= 7
http://ke.kabupro.jp/code/1380.htm i= 8
http://ke.kabupro.jp/code/1381.htm i= 9
http://ke.kabupro.jp/code/1382.htm i= 10
http://ke.kabupro.jp/code/1383.htm i= 11
http://ke.kabupro.jp/code/1384.htm i= 12
http://ke.kabupro.jp/code/1400.htm i= 13
http://ke.kabupro.jp/code/1401.htm i= 14
http://ke.kabupro.jp/code/1407.htm i= 15
http://ke.kabupro.jp/code/1413.htm i= 16
http://ke.kabupro.jp/code/1414.htm i= 17
http://ke.kabupro.jp/code/1417.htm i= 18
http://ke.kabupro.jp/co

http://ke.kabupro.jp/code/1850.htm i= 115
http://ke.kabupro.jp/code/1852.htm i= 116
http://ke.kabupro.jp/code/1853.htm i= 117
http://ke.kabupro.jp/code/1860.htm i= 118
http://ke.kabupro.jp/code/1861.htm i= 119
http://ke.kabupro.jp/code/1866.htm i= 120
http://ke.kabupro.jp/code/1867.htm i= 121
取込エラー
Invalid dictionary construct: [/'Type', /'Font', /'BaseFont', /b"b'", /"\\x82l\\x82r\\x96\\xbe\\x92\\xa9'", /'Subtype', /'Type0', /'DescendantFonts', [<PDFObjRef:16>], /'Encoding', /'UniJIS-UTF16-H']


http://ke.kabupro.jp/code/1869.htm i= 122
http://ke.kabupro.jp/code/1870.htm i= 123
取込エラー
Invalid dictionary construct: [/'Type', /'Font', /'BaseFont', /b"b'", /"\\x82l\\x82r\\x96\\xbe\\x92\\xa9'", /'Subtype', /'Type0', /'DescendantFonts', [<PDFObjRef:17>], /'Encoding', /'UniJIS-UTF16-H']


http://ke.kabupro.jp/code/1871.htm i= 124
取込エラー
Invalid dictionary construct: [/'Type', /'Font', /'BaseFont', /b"b'", /"\\x82l\\x82r\\x96\\xbe\\x92\\xa9'", /'Subtype', /'Type0', /'DescendantFonts', [<PDFObj

http://ke.kabupro.jp/code/2173.htm i= 234
取込エラー
Invalid dictionary construct: [/'Type', /'Font', /'BaseFont', /b"b'", /"\\x82l\\x82r\\x96\\xbe\\x92\\xa9'", /'Subtype', /'Type0', /'DescendantFonts', [<PDFObjRef:12>], /'Encoding', /'UniJIS-UTF16-H']


http://ke.kabupro.jp/code/2174.htm i= 235
取込エラー
only algorithm code 1 and 2 are supported


http://ke.kabupro.jp/code/2175.htm i= 236
取込エラー
only algorithm code 1 and 2 are supported


http://ke.kabupro.jp/code/2176.htm i= 237
http://ke.kabupro.jp/code/2178.htm i= 238
http://ke.kabupro.jp/code/2179.htm i= 239
http://ke.kabupro.jp/code/2180.htm i= 240
http://ke.kabupro.jp/code/2181.htm i= 241
取込エラー
Invalid dictionary construct: [/'Type', /'Font', /'BaseFont', /b"b'", /"\\x82l\\x82r\\x96\\xbe\\x92\\xa9'", /'Subtype', /'Type0', /'DescendantFonts', [<PDFObjRef:12>], /'Encoding', /'UniJIS-UTF16-H']


http://ke.kabupro.jp/code/2183.htm i= 242
http://ke.kabupro.jp/code/2185.htm i= 243
取込エラー
Invalid dictionary construct: [/'Type', /'Font', /'BaseFon

http://ke.kabupro.jp/code/2398.htm i= 334
http://ke.kabupro.jp/code/2402.htm i= 335
http://ke.kabupro.jp/code/2404.htm i= 336
http://ke.kabupro.jp/code/2406.htm i= 337
取込エラー
only algorithm code 1 and 2 are supported


http://ke.kabupro.jp/code/2408.htm i= 338
http://ke.kabupro.jp/code/2410.htm i= 339
取込エラー
Invalid dictionary construct: [/'Type', /'Font', /'BaseFont', /b"b'", /"\\x82l\\x82r\\x96\\xbe\\x92\\xa9'", /'Subtype', /'Type0', /'DescendantFonts', [<PDFObjRef:12>], /'Encoding', /'UniJIS-UTF16-H']


http://ke.kabupro.jp/code/2411.htm i= 340
取込エラー
only algorithm code 1 and 2 are supported


http://ke.kabupro.jp/code/2412.htm i= 341
http://ke.kabupro.jp/code/2413.htm i= 342
取込エラー
Invalid dictionary construct: [/'Type', /'Font', /'BaseFont', /b"b'", /"\\x82l\\x82r\\x96\\xbe\\x92\\xa9'", /'Subtype', /'Type0', /'DescendantFonts', [<PDFObjRef:17>], /'Encoding', /'UniJIS-UTF16-H']


http://ke.kabupro.jp/code/2415.htm i= 343
http://ke.kabupro.jp/code/2418.htm i= 344
http://ke.kabupro.jp/c

http://ke.kabupro.jp/code/2692.htm i= 430
http://ke.kabupro.jp/code/2693.htm i= 431
http://ke.kabupro.jp/code/2694.htm i= 432
http://ke.kabupro.jp/code/2695.htm i= 433
http://ke.kabupro.jp/code/2698.htm i= 434
http://ke.kabupro.jp/code/2700.htm i= 435
http://ke.kabupro.jp/code/2702.htm i= 436
http://ke.kabupro.jp/code/2705.htm i= 437
http://ke.kabupro.jp/code/2706.htm i= 438
http://ke.kabupro.jp/code/2708.htm i= 439
http://ke.kabupro.jp/code/2715.htm i= 440
http://ke.kabupro.jp/code/2721.htm i= 441
取込エラー
Invalid dictionary construct: [/'Type', /'Font', /'BaseFont', /b"b'", /"\\x82l\\x82r\\x96\\xbe\\x92\\xa9'", /'Subtype', /'Type0', /'DescendantFonts', [<PDFObjRef:12>], /'Encoding', /'UniJIS-UTF16-H']


http://ke.kabupro.jp/code/2722.htm i= 442
http://ke.kabupro.jp/code/2726.htm i= 443
http://ke.kabupro.jp/code/2729.htm i= 444
http://ke.kabupro.jp/code/2730.htm i= 445
http://ke.kabupro.jp/code/2733.htm i= 446
取込エラー
Invalid dictionary construct: [/'Type', /'Font', /'BaseFont', /b"b'", /"

...

http://ke.kabupro.jp/code/8747.htm i= 3347
http://ke.kabupro.jp/code/8750.htm i= 3348
http://ke.kabupro.jp/code/8766.htm i= 3349
取込エラー
Invalid dictionary construct: [/'Type', /'Font', /'BaseFont', /b"b'", /"\\x82l\\x82r\\x96\\xbe\\x92\\xa9'", /'Subtype', /'Type0', /'DescendantFonts', [<PDFObjRef:12>], /'Encoding', /'UniJIS-UTF16-H']


http://ke.kabupro.jp/code/8769.htm i= 3350
取込エラー
Invalid dictionary construct: [/'Type', /'Font', /'BaseFont', /b"b'", /"\\x82l\\x82r\\x83S\\x83V\\x83b\\x83N'", /'Subtype', /'Type0', /'DescendantFonts', [<PDFObjRef:12>], /'Encoding', /'UniJIS-UTF16-H']


http://ke.kabupro.jp/code/8771.htm i= 3351
http://ke.kabupro.jp/code/8772.htm i= 3352
http://ke.kabupro.jp/code/8783.htm i= 3353
http://ke.kabupro.jp/code/8789.htm i= 3354
http://ke.kabupro.jp/code/8793.htm i= 3355
http://ke.kabupro.jp/code/8795.htm i= 3356
http://ke.kabupro.jp/code/8798.htm i= 3357
http://ke.kabupro.jp/code/8801.htm i= 3358
http://ke.kabupro.jp/code/8802.htm i= 3359
取込エラー
Invalid diction

http://ke.kabupro.jp/code/9057.htm i= 3455
http://ke.kabupro.jp/code/9058.htm i= 3456
http://ke.kabupro.jp/code/9059.htm i= 3457
http://ke.kabupro.jp/code/9060.htm i= 3458
http://ke.kabupro.jp/code/9062.htm i= 3459
http://ke.kabupro.jp/code/9063.htm i= 3460
取込エラー
Invalid dictionary construct: [/'Type', /'Font', /'BaseFont', /b"b'", /"\\x82l\\x82r\\x96\\xbe\\x92\\xa9'", /'Subtype', /'Type0', /'DescendantFonts', [<PDFObjRef:12>], /'Encoding', /'UniJIS-UTF16-H']


http://ke.kabupro.jp/code/9064.htm i= 3461
http://ke.kabupro.jp/code/9065.htm i= 3462
http://ke.kabupro.jp/code/9066.htm i= 3463
http://ke.kabupro.jp/code/9067.htm i= 3464
取込エラー
No objects to concatenate


http://ke.kabupro.jp/code/9068.htm i= 3465
http://ke.kabupro.jp/code/9069.htm i= 3466
http://ke.kabupro.jp/code/9070.htm i= 3467
http://ke.kabupro.jp/code/9072.htm i= 3468
http://ke.kabupro.jp/code/9073.htm i= 3469
http://ke.kabupro.jp/code/9074.htm i= 3470
http://ke.kabupro.jp/code/9075.htm i= 3471
http://ke.kabupro.jp/code/9

http://ke.kabupro.jp/code/9404.htm i= 3566
取込エラー
only algorithm code 1 and 2 are supported


http://ke.kabupro.jp/code/9405.htm i= 3567
http://ke.kabupro.jp/code/9407.htm i= 3568
http://ke.kabupro.jp/code/9408.htm i= 3569
取込エラー
Invalid dictionary construct: [/'Type', /'Font', /'BaseFont', /b"b'", /"\\x82l\\x82r\\x96\\xbe\\x92\\xa9'", /'Subtype', /'Type0', /'DescendantFonts', [<PDFObjRef:12>], /'Encoding', /'UniJIS-UTF16-H']


http://ke.kabupro.jp/code/9409.htm i= 3570
http://ke.kabupro.jp/code/9412.htm i= 3571
http://ke.kabupro.jp/code/9413.htm i= 3572
http://ke.kabupro.jp/code/9414.htm i= 3573
http://ke.kabupro.jp/code/9416.htm i= 3574
http://ke.kabupro.jp/code/9417.htm i= 3575
http://ke.kabupro.jp/code/9418.htm i= 3576
http://ke.kabupro.jp/code/9419.htm i= 3577
取込エラー
Invalid dictionary construct: [/'Type', /'Font', /'BaseFont', /b"b'", /"\\x82l\\x82r\\x96\\xbe\\x92\\xa9'", /'Subtype', /'Type0', /'DescendantFonts', [<PDFObjRef:12>], /'Encoding', /'UniJIS-UTF16-H']


http://ke.kabupro.

http://ke.kabupro.jp/code/9678.htm i= 3668
http://ke.kabupro.jp/code/9679.htm i= 3669
取込エラー
Invalid dictionary construct: [/'Type', /'Font', /'BaseFont', /b"b'", /"\\x82l\\x82r\\x96\\xbe\\x92\\xa9'", /'Subtype', /'Type0', /'DescendantFonts', [<PDFObjRef:12>], /'Encoding', /'UniJIS-UTF16-H']


http://ke.kabupro.jp/code/9682.htm i= 3670
http://ke.kabupro.jp/code/9684.htm i= 3671
取込エラー
Invalid dictionary construct: [/'Type', /'Font', /'BaseFont', /b"b'", /"\\x82l\\x82r\\x96\\xbe\\x92\\xa9'", /'Subtype', /'Type0', /'DescendantFonts', [<PDFObjRef:12>], /'Encoding', /'UniJIS-UTF16-H']


http://ke.kabupro.jp/code/9685.htm i= 3672
http://ke.kabupro.jp/code/9686.htm i= 3673
http://ke.kabupro.jp/code/9687.htm i= 3674
http://ke.kabupro.jp/code/9691.htm i= 3675
http://ke.kabupro.jp/code/9692.htm i= 3676
取込エラー
Invalid dictionary construct: [/'Type', /'Font', /'BaseFont', /b"b'", /"\\x82l\\x82r\\x96\\xbe\\x92\\xa9'", /'Subtype', /'Type0', /'DescendantFonts', [<PDFObjRef:12>], /'Encoding', /'UniJIS-U

http://ke.kabupro.jp/code/9945.htm i= 3797
http://ke.kabupro.jp/code/9946.htm i= 3798
http://ke.kabupro.jp/code/9948.htm i= 3799
取込エラー
Invalid dictionary construct: [/'Type', /'Font', /'BaseFont', /b"b'", /"\\x82l\\x82r\\x96\\xbe\\x92\\xa9'", /'Subtype', /'Type0', /'DescendantFonts', [<PDFObjRef:12>], /'Encoding', /'UniJIS-UTF16-H']


http://ke.kabupro.jp/code/9950.htm i= 3800
http://ke.kabupro.jp/code/9955.htm i= 3801
http://ke.kabupro.jp/code/9956.htm i= 3802
http://ke.kabupro.jp/code/9959.htm i= 3803
http://ke.kabupro.jp/code/9960.htm i= 3804
http://ke.kabupro.jp/code/9962.htm i= 3805
http://ke.kabupro.jp/code/9964.htm i= 3806
取込エラー
only algorithm code 1 and 2 are supported


http://ke.kabupro.jp/code/9966.htm i= 3807
http://ke.kabupro.jp/code/9967.htm i= 3808
http://ke.kabupro.jp/code/9969.htm i= 3809
http://ke.kabupro.jp/code/9972.htm i= 3810
http://ke.kabupro.jp/code/9973.htm i= 3811
http://ke.kabupro.jp/code/9974.htm i= 3812
http://ke.kabupro.jp/code/9976.htm i= 3813
http://ke.k

In [5]:
#手順1：tabulaで取り込む銘柄コードの表をcsvへ書込み
#かなり後で使用する
df_for_tabula = pd.DataFrame()
df_for_tabula['code'] = tabula_dict.keys()
df_for_tabula['error'] = tabula_dict.values()

df_for_tabula.to_csv('temp_file/全銘柄PDFデータ（手順1_tabula用銘柄コード）.csv', mode='w', index=False, encoding="shift-jis", errors='none')

df_for_tabula

,code,error
0,1333,"Invalid dictionary construct: [/'Type', /'Font..."
1,1438,"Invalid dictionary construct: [/'Type', /'Font..."
2,1447,"Invalid dictionary construct: [/'Type', /'Font..."
3,1448,"Invalid dictionary construct: [/'Type', /'Font..."
4,1451,"Invalid dictionary construct: [/'Type', /'Font..."
...,...,...
821,9964,only algorithm code 1 and 2 are supported
822,9978,"Invalid dictionary construct: [/'Type', /'Font..."
823,9983,"Invalid dictionary construct: [/'Type', /'Font..."
824,9984,only algorithm code 1 and 2 are supported


In [7]:
#手順1：camelotで取得した表の確認
df_camelot1 = df_camelot1.reindex(columns=['code', '流動資産合計', '現金及び預金', '現金預金', '有形固定資産', '有形固定資産合計', 
                         '投資有価証券', '投資その他の資産合計', '流動負債合計', '固定負債合計', ])
df_camelot1

,code,流動資産合計,現金及び預金,現金預金,有形固定資産,有形固定資産合計,投資有価証券,投資その他の資産合計,流動負債合計,固定負債合計
0,1301,"86,649","7,100",NaN,,"15,715","11,682","13,547","39,189","37,166"
0,1332,"231,751","10,986",NaN,,"147,428","38,076","86,443","153,467","134,021"
0,1352,"12,137","4,019",NaN,,"26,593",345,"2,390","11,866","21,666"
0,1375,"10,202",NaN,NaN,"18,031",NaN,NaN,NaN,"7,614",NaN
0,1376,"28,828,145","3,447,948",NaN,,"6,627,416",NaN,"2,829,597","16,514,107","1,726,482"
...,...,...,...,...,...,...,...,...,...,...
0,9991,"71,446",20.8,NaN,△1.4,"23,439",△0.0,"7,290","37,959","5,094"
0,9994,"32,537","8,664",NaN,,"11,459",894,"11,165","28,133","5,102"
0,9995,"28,864","3,395",NaN,,"1,537","2,131","3,112","10,263",888
0,9996,"18,787,671","2,448,882",NaN,,"5,577,535","6,167,809","7,689,024","7,800,804","345,038"


In [8]:
#手順1：camelotで取得した表をcsvへ書込み
df_camelot1.to_csv('temp_file/全銘柄PDFデータ（手順1_camelot加工前）.csv', mode='w', index=False, encoding="shift-jis", errors='none')

In [9]:
#手順1：camelotで取得した表をcsvから読み込み、余分な文字を消す
df_camelot2 = pd.read_csv('temp_file/全銘柄PDFデータ（手順1_camelot加工前）.csv', encoding=("cp932"), header=0)

replace_feature_list = list(df_camelot2.select_dtypes(include=object).columns.values)
replace_list = [',', '△', '―', '-', '，', '\n', ' ',
                '※1', '※2', '※3', '※4', '※5', '※6', '※7', '※8', '※9',
                '※１', '※２', '※３', '※４', '※５', '※６', '※７', '※８', '※９']
for i in replace_feature_list:
    for j in replace_list:
        df_camelot2[i] = df_camelot2[i].str.replace(j, '')
df_camelot2

,code,流動資産合計,現金及び預金,現金預金,有形固定資産,有形固定資産合計,投資有価証券,投資その他の資産合計,流動負債合計,固定負債合計
0,1301,86649,7100,NaN,NaN,15715,11682,13547,39189,37166
1,1332,231751,10986,NaN,NaN,147428,38076,86443,153467,134021
2,1352,12137,4019,NaN,NaN,26593,345,2390,11866,21666
3,1375,10202,NaN,NaN,18031,NaN,NaN,NaN,7614,NaN
4,1376,28828145,3447948,NaN,NaN,6627416,NaN,2829597,16514107,1726482
...,...,...,...,...,...,...,...,...,...,...
2990,9991,71446,20.8,NaN,1.4,23439,0.0,7290,37959,5094
2991,9994,32537,8664,NaN,NaN,11459,894,11165,28133,5102
2992,9995,28864,3395,NaN,NaN,1537,2131,3112,10263,888
2993,9996,18787671,2448882,NaN,NaN,5577535,6167809,7689024,7800804,345038


In [10]:
#手順1：十分に取り込めていない銘柄コードについて、
#①再抽出する銘柄コードの抽出と、②削除する銘柄コードの抽出
camelot_re_dict = {}
camelot_drop_list = []
for i in range(0, len(df_camelot2)):
    #①再抽出する銘柄コードの抽出
    if len(df_camelot2.iloc[i, :].value_counts()) == 1:
        camelot_re_dict[df_camelot2.iloc[i, 0]] = 're1'
        print('code=', df_camelot2.iloc[i, 0], '再抽出：資料が左に細くて1つも取り込めない')
    elif df_camelot2.iloc[i, 9] == '非流動負債合計':
        camelot_re_dict[df_camelot2.iloc[i, 0]] = 're2'
        print('code=', df_camelot2.iloc[i, 0], '再抽出：固定負債合計の値が非流動負債合計')
        
for i in range(0, len(df_camelot2)):    
    #②削除する銘柄コードの抽出
    if len(df_camelot2.iloc[i, :].value_counts()) <= 3:
        camelot_drop_list.append(df_camelot2.iloc[i, 0])
        print('code=', df_camelot2.iloc[i, 0], '削除：特徴量が2つ以下')
    elif df_camelot2.iloc[i, :].str.contains('%').sum() !=0:
        camelot_drop_list.append(df_camelot2.iloc[i, 0])
        print('code=', df_camelot2.iloc[i, 0], '削除：%を含む')
    elif df_camelot2.iloc[i, 9] == '非流動負債合計':
        camelot_drop_list.append(df_camelot2.iloc[i, 0])
        print('code=', df_camelot2.iloc[i, 0], '削除：固定負債合計の値が非流動負債合計')

code= 3402 再抽出：固定負債合計の値が非流動負債合計
code= 4183 再抽出：固定負債合計の値が非流動負債合計
code= 6232 再抽出：資料が左に細くて1つも取り込めない
code= 6361 再抽出：固定負債合計の値が非流動負債合計
code= 7203 再抽出：固定負債合計の値が非流動負債合計
code= 7605 再抽出：資料が左に細くて1つも取り込めない
code= 8160 再抽出：資料が左に細くて1つも取り込めない
code= 8399 再抽出：資料が左に細くて1つも取り込めない
code= 8746 再抽出：資料が左に細くて1つも取り込めない
code= 9201 再抽出：固定負債合計の値が非流動負債合計
code= 9519 再抽出：固定負債合計の値が非流動負債合計
code= 9678 再抽出：資料が左に細くて1つも取り込めない
code= 9719 再抽出：固定負債合計の値が非流動負債合計
code= 3245 削除：%を含む
code= 3402 削除：固定負債合計の値が非流動負債合計
code= 4095 削除：特徴量が2つ以下
code= 4183 削除：固定負債合計の値が非流動負債合計
code= 4519 削除：特徴量が2つ以下
code= 4689 削除：特徴量が2つ以下
code= 4755 削除：特徴量が2つ以下
code= 5202 削除：特徴量が2つ以下
code= 5464 削除：特徴量が2つ以下
code= 6098 削除：特徴量が2つ以下
code= 6232 削除：特徴量が2つ以下
code= 6361 削除：固定負債合計の値が非流動負債合計
code= 6417 削除：特徴量が2つ以下
code= 6594 削除：特徴量が2つ以下
code= 6697 削除：特徴量が2つ以下
code= 6758 削除：特徴量が2つ以下
code= 6981 削除：特徴量が2つ以下
code= 7150 削除：特徴量が2つ以下
code= 7157 削除：特徴量が2つ以下
code= 7167 削除：特徴量が2つ以下
code= 7181 削除：特徴量が2つ以下
code= 7182 削除：特徴量が2つ以下
code= 7184 削除：特徴量が2つ以下
code= 7189 削除：特徴量が2つ以下
code= 

In [11]:
#手順1：削除する銘柄コードの削除
for i in camelot_drop_list:
    df_camelot2.drop(df_camelot2.index[df_camelot2['code'] == i], inplace=True)
df_camelot2

,code,流動資産合計,現金及び預金,現金預金,有形固定資産,有形固定資産合計,投資有価証券,投資その他の資産合計,流動負債合計,固定負債合計
0,1301,86649,7100,NaN,NaN,15715,11682,13547,39189,37166
1,1332,231751,10986,NaN,NaN,147428,38076,86443,153467,134021
2,1352,12137,4019,NaN,NaN,26593,345,2390,11866,21666
3,1375,10202,NaN,NaN,18031,NaN,NaN,NaN,7614,NaN
4,1376,28828145,3447948,NaN,NaN,6627416,NaN,2829597,16514107,1726482
...,...,...,...,...,...,...,...,...,...,...
2990,9991,71446,20.8,NaN,1.4,23439,0.0,7290,37959,5094
2991,9994,32537,8664,NaN,NaN,11459,894,11165,28133,5102
2992,9995,28864,3395,NaN,NaN,1537,2131,3112,10263,888
2993,9996,18787671,2448882,NaN,NaN,5577535,6167809,7689024,7800804,345038


In [12]:
#手順1：camelotで取得した表（加工・削除後）をcsvへ書込み
df_camelot2.to_csv('temp_file/全銘柄PDFデータ（手順1_camelot完成）.csv', mode='w', index=False, encoding="shift-jis", errors='none')

In [13]:
#手順2：camelotで再抽出する銘柄コードと、空データフレームの作成
codes_camelot_re = list(camelot_re_dict.keys())
df_camelot_re1 = pd.DataFrame()

In [14]:
#手順2：camelotで再抽出
def get_pdf_camelot_re(start_url, code, frag):
    time.sleep(1)
    response = requests.get(start_url)
    soup = BeautifulSoup(response.content, 'html.parser')

    table = soup.find('table', attrs={'class': 'Quote'})
    #訂正があった場合でも、訂正前のPDFを取得する
    for i in range(0, 5):
        if '訂正' in table.find_all('td', text = re.compile('.*決算短信.*'))[i].text\
        or '補足' in table.find_all('td', text = re.compile('.*決算短信.*'))[i].text:
            pass
        else:
            target_pdf = table.find_all('td', text = re.compile('.*決算短信.*'))[i].find('a').get('href')
            break
                
    try:
        tables = camelot.read_pdf(target_pdf, flavor='stream', pages = '4-end', table_areas=['0,800,700,0'])
        if frag == 're1':
            get_features_camelot_re1(tables, code)
        else:
            get_features_camelot_re2(tables, code)
    except Exception as e:
        print('取込エラー')
        print(e)
        print('\n')
        pass

#手順2：対応①　camelot「固定負債合計の値が非流動負債合計」PDFデータの取込み
def get_features_camelot_re1(tables, code):
    feature_list = ['流動資産合計', '現金及び預金', '現金預金', '有形固定資産', '有形固定資産合計', '投資有価証券',
                    '投資その他の資産合計', '流動負債合計', '固定負債合計']
    feature_remove_list = []
    df_elements = []
    for table in tables:
        df_tbl = table.df  
        for feature in feature_list:
            if len(df_tbl[df_tbl.iloc[:,0] == feature]) == 1:
                #資産と負債が違うテーブルのため、df_element1の列がずれる場合がある
                #特徴量と値だけの1行2列のデータフレームを作成
                df_element1 = df_tbl[df_tbl.iloc[:,0] == feature].iloc[:, [0,-1]]
                #【左から2列目に値が入っているPDFのため、その対応をする】
                if df_element1.iloc[0, 1] == '':
                    df_element1 = df_tbl[df_tbl.iloc[:,0] == feature].iloc[:, [0,-2]]
                #後のconcatで列がずれないようにするため、値の列の名前を1に統一する
                df_element2 = df_element1.rename(columns={df_element1.columns[1]: 1})
                df_elements.append(df_element2)
                #資産と負債のテーブルが2つある銘柄がある（連結と単独のテーブル）
                #同じ特徴量を2回取得することを避けるため、一度入れた特徴量は削除リストに登録
                feature_remove_list.append(feature)
        #テーブル処理が1つ終わったら、削除リストに登録された特徴量を元のリストから削除する
        if len(feature_remove_list) != 0:
            for feature_remove in feature_remove_list:
                feature_list.remove(feature_remove)
            feature_remove_list = []
    df_all1 = pd.concat(df_elements).T
    df_all2 = pd.DataFrame([list(df_all1.iloc[1,:])], columns=df_all1.iloc[0,:],)  
    df_all2['code'] = code
    dfs_concat_camelot_re(df_all2)

    
#手順2：対応②　camelot「固定負債合計の値が非流動負債合計」PDFデータの再抽出
def get_features_camelot_re2(tables, code):
#【現金及び預金→現金及び現金同等物、固定負債合計→非流動負債合計に変更】
    feature_list = ['流動資産合計', '現金及び現金同等物', '現金預金', '有形固定資産', '有形固定資産合計', '投資有価証券',
                    '投資その他の資産合計', '流動負債合計', '非流動負債合計']
    feature_remove_list = []
    df_elements = []
    for table in tables:
        df_tbl = table.df  
        for feature in feature_list:
            if len(df_tbl[df_tbl.iloc[:,0] == feature]) == 1:
                #資産と負債が違うテーブルのため、df_element1の列がずれる場合がある
                #特徴量と値だけの1行2列のデータフレームを作成
                df_element1 = df_tbl[df_tbl.iloc[:,0] == feature].iloc[:, [0,-1]]
                #【左から2列目に値が入っているPDFのため、その対応をする】
                if df_element1.iloc[0, 1] == '':
                    df_element1 = df_tbl[df_tbl.iloc[:,0] == feature].iloc[:, [0,-2]]
                #後のconcatで列がずれないようにするため、値の列の名前を1に統一する
                df_element2 = df_element1.rename(columns={df_element1.columns[1]: 1})
                df_elements.append(df_element2)
                #資産と負債のテーブルが2つある銘柄がある（連結と単独のテーブル）
                #同じ特徴量を2回取得することを避けるため、一度入れた特徴量は削除リストに登録
                feature_remove_list.append(feature)
        #テーブル処理が1つ終わったら、削除リストに登録された特徴量を元のリストから削除する
        if len(feature_remove_list) != 0:
            for feature_remove in feature_remove_list:
                feature_list.remove(feature_remove)
            feature_remove_list = []
    df_all1 = pd.concat(df_elements).T
    df_all2 = pd.DataFrame([list(df_all1.iloc[1,:])], columns=df_all1.iloc[0,:],)\
    .rename(columns={'現金及び現金同等物': '現金及び預金'}).rename(columns={'非流動負債合計': '固定負債合計'}) 
    df_all2['code'] = code
    dfs_concat_camelot_re(df_all2)
    
    
def dfs_concat_camelot_re(df_all2):
    global df_camelot_re1
    df_camelot_re1 = pd.concat([df_camelot_re1, df_all2], axis=0)

In [15]:
#手順2：camelotで再抽出をPDFから特徴量をDataFrameへ記入　
for i in range(0, len(codes_camelot_re)):
    code = codes_camelot_re[i]
    frag = camelot_re_dict[code]
    start_url = 'http://ke.kabupro.jp/code/{}.htm'.format(code)
    print(start_url, 'i=', i)
    get_pdf_camelot_re(start_url, code, frag)

http://ke.kabupro.jp/code/3402.htm i= 0
http://ke.kabupro.jp/code/4183.htm i= 1
http://ke.kabupro.jp/code/6232.htm i= 2
http://ke.kabupro.jp/code/6361.htm i= 3
http://ke.kabupro.jp/code/7203.htm i= 4
http://ke.kabupro.jp/code/7605.htm i= 5
http://ke.kabupro.jp/code/8160.htm i= 6
http://ke.kabupro.jp/code/8399.htm i= 7
http://ke.kabupro.jp/code/8746.htm i= 8
http://ke.kabupro.jp/code/9201.htm i= 9
http://ke.kabupro.jp/code/9519.htm i= 10
http://ke.kabupro.jp/code/9678.htm i= 11
http://ke.kabupro.jp/code/9719.htm i= 12


In [16]:
#手順2：camelotで再取得した表の確認
df_camelot_re1 = df_camelot_re1.reindex(columns=['code', '流動資産合計', '現金及び預金', '現金預金', '有形固定資産', '有形固定資産合計', 
                         '投資有価証券', '投資その他の資産合計', '流動負債合計', '固定負債合計', ])
df_camelot_re1

,code,流動資産合計,現金及び預金,現金預金,有形固定資産,有形固定資産合計,投資有価証券,投資その他の資産合計,流動負債合計,固定負債合計
0,3402,"1,181,039","236,354",NaN,"998,358",NaN,,NaN,"681,378","845,186"
0,4183,"787,572","195,987",NaN,"455,749",NaN,,NaN,"466,305","409,663"
0,6232,"3,257,717","1,891,731",NaN,NaN,NaN,"646,888","675,887","432,941","3,346"
0,6361,"468,136","136,084",NaN,"160,722",NaN,,NaN,"291,212","82,087"
0,7203,"22,776,800","5,100,857",NaN,,"11,411,153",,NaN,"21,460,466","16,518,344"
0,7605,"13,909,184","6,247,882",NaN,,"13,160,081",NaN,NaN,NaN,NaN
0,8160,"20,354","16,002",NaN,,"14,779","2,825","8,027","16,515","6,087"
0,8399,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN
0,8746,"7,319,602","2,334,471",NaN,,"388,267","1,628,841","2,056,000","4,204,103","44,459"
0,9201,"567,816","408,335",NaN,,"1,045,413","(C),(c) \nその他の金融資産",NaN,"476,893","648,850"


In [17]:
#手順2：camelotで再取得した表をcsvへ書込み
df_camelot_re1.to_csv('temp_file/全銘柄PDFデータ（手順2_camelot_re加工前）.csv', mode='w', 
                      index=False, encoding="shift-jis", errors='none')

In [18]:
#手順2：camelotで再取得した表の余分な文字を消す
df_camelot_re2 = pd.read_csv('temp_file/全銘柄PDFデータ（手順2_camelot_re加工前）.csv', encoding=("cp932"), header=0)
replace_feature_list = list(df_camelot_re2.select_dtypes(include=object).columns.values)
replace_list = [',', '△', '―', '-', '，', '\n', ' ',
                '※1', '※2', '※3', '※4', '※5', '※6', '※7', '※8', '※9',
                '※１', '※２', '※３', '※４', '※５', '※６', '※７', '※８', '※９', '\D']

for i in replace_feature_list:
    for j in replace_list:
        df_camelot_re2[i] = df_camelot_re2[i].str.replace(j, '')
df_camelot_re2


,code,流動資産合計,現金及び預金,現金預金,有形固定資産,有形固定資産合計,投資有価証券,投資その他の資産合計,流動負債合計,固定負債合計
0,3402,1181039,236354,NaN,998358,NaN,NaN,NaN,681378,845186
1,4183,787572,195987,NaN,455749,NaN,NaN,NaN,466305,409663
2,6232,3257717,1891731,NaN,NaN,NaN,646888,675887,432941,3346
3,6361,468136,136084,NaN,160722,NaN,NaN,NaN,291212,82087
4,7203,22776800,5100857,NaN,NaN,11411153,NaN,NaN,21460466,16518344
5,7605,13909184,6247882,NaN,NaN,13160081,NaN,NaN,NaN,NaN
6,8160,20354,16002,NaN,NaN,14779,2825,8027,16515,6087
7,8399,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8746,7319602,2334471,NaN,NaN,388267,1628841,2056000,4204103,44459
9,9201,567816,408335,NaN,NaN,1045413,,NaN,476893,648850


In [19]:
#手順2：camelotで再取得した銘柄のうち、十分に取り込めていない銘柄コードの抽出と削除
camelot_re_drop_list = []

#削除する銘柄コードの抽出        
for i in range(0, len(df_camelot_re2)):    
    #②削除する銘柄コードの抽出
    if len(df_camelot_re2.iloc[i, :].value_counts()) <= 3:
        camelot_re_drop_list.append(df_camelot_re2.iloc[i, 0])
        print('code=', df_camelot_re2.iloc[i, 0], '削除：特徴量が2つ以下')
        
#削除する銘柄コードの削除
for i in camelot_re_drop_list:
    df_camelot_re2.drop(df_camelot_re2.index[df_camelot_re2['code'] == i], inplace=True)
df_camelot_re2

code= 8399 削除：特徴量が2つ以下


,code,流動資産合計,現金及び預金,現金預金,有形固定資産,有形固定資産合計,投資有価証券,投資その他の資産合計,流動負債合計,固定負債合計
0,3402,1181039,236354,NaN,998358,NaN,NaN,NaN,681378,845186
1,4183,787572,195987,NaN,455749,NaN,NaN,NaN,466305,409663
2,6232,3257717,1891731,NaN,NaN,NaN,646888,675887,432941,3346
3,6361,468136,136084,NaN,160722,NaN,NaN,NaN,291212,82087
4,7203,22776800,5100857,NaN,NaN,11411153,NaN,NaN,21460466,16518344
5,7605,13909184,6247882,NaN,NaN,13160081,NaN,NaN,NaN,NaN
6,8160,20354,16002,NaN,NaN,14779,2825,8027,16515,6087
8,8746,7319602,2334471,NaN,NaN,388267,1628841,2056000,4204103,44459
9,9201,567816,408335,NaN,NaN,1045413,,NaN,476893,648850
10,9519,46699,19406,NaN,104148,NaN,NaN,NaN,13375,182306


In [20]:
#手順2：camelotで再取得した表（加工・削除後）をcsvへ書込み
df_camelot_re2.to_csv('temp_file/全銘柄PDFデータ（手順2_camelot_re完成）.csv', mode='w',
                      index=False, encoding="shift-jis", errors='none')

In [21]:
#手順3：tabulaの利用
def get_pdf_tabula(start_url_tabula, code_tabula):
    response = requests.get(start_url_tabula)
    soup = BeautifulSoup(response.content, 'html.parser')
    if 'リクエストされたページがみつかりませんでした' in soup.text:
        print('東証以外の銘柄のため、PDF見つからず')
        pass
    else:
        table = soup.find('table', attrs={'class': 'Quote'})  
        #訂正があった場合でも、訂正前のPDFを取得する
        for i in range(0, 5):
            if '訂正' in table.find_all('td', text = re.compile('.*決算短信.*'))[i].text\
            or '補足' in table.find_all('td', text = re.compile('.*決算短信.*'))[i].text:
                pass
            else:
                target_pdf = table.find_all('td', text = re.compile('.*決算短信.*'))[i].find('a').get('href')
                break

        try:
            '''
            area (list of float, list of list of float, optional) –
            Portion of the page to analyze(top,left,bottom,right). Default is entire page.
            '''
            df_tbls = tabula.read_pdf(target_pdf, pages = 'all', stream=True, silent=True)
            get_features_tabula(df_tbls, code_tabula)           
        except Exception as e:
            error_dict[code_tabula] = e
            print('取込エラー')
            print(e)
            print('\n')
            pass

def get_features_tabula(df_tbls, code_tabula):
    feature_list = ['流動資産合計', '現金及び預金', '現金預金', '有形固定資産', '有形固定資産合計', '投資有価証券', 
                '投資その他の資産合計', '流動負債合計', '固定負債合計']
    df_elements = []
    feature_remove_list = []
    count_tables = 0
    for df_tbl in df_tbls:
        count = 0
        for feature in feature_list:
            if len(df_tbl[df_tbl.iloc[:,0] == feature]) == 1:
                #資産と負債が違うテーブルのため、df_element1の列がずれる場合がある
                #特徴量と値だけの1行2列のデータフレームを作成
                df_element1 = df_tbl[df_tbl.iloc[:,0] == feature].iloc[:, [0,-1]]
                #後のconcatで列がずれないようにするため、値の列の名前が違うため1に統一する
                df_element2 = df_element1.rename(columns={df_element1.columns[1]: 1})
                df_elements.append(df_element2)
                #資産と負債のテーブルが2つある銘柄がある（連結と単独のテーブル）
                #同じ特徴量を2回取得することを避けるため、一度入れた特徴量は削除リストに登録
                feature_remove_list.append(feature)
        #テーブル処理が1つ終わったら、削除リストに登録された特徴量を元のリストから削除する
        if len(feature_remove_list) != 0:
            for feature_remove in feature_remove_list:
                feature_list.remove(feature_remove)
            feature_remove_list = []

    df_all1 = pd.concat(df_elements).T
    df_all2 = pd.DataFrame([list(df_all1.iloc[1,:])], columns=df_all1.iloc[0,:],)  
    df_all2['code'] = code_tabula
    dfs_concat_tabula(df_all2)
    
    
def dfs_concat_tabula(df_all2):
    global df_tabula1
    df_tabula1 = pd.concat([df_tabula1, df_all2], axis=0)


In [22]:
#手順3：取り込む銘柄コードと、空データフレームの作成
#PDFの表抽出には、camelotとtabulaがある。それぞれ抽出できない銘柄がいくつか存在する。
#camelotの方が抽出できる銘柄が多いので、camelotで抽出後、残りをtabulaで抽出する。

codes_tabula = pd.read_csv('temp_file/全銘柄PDFデータ（手順1_tabula用銘柄コード）.csv', 
                           encoding=("cp932"), header=0).iloc[:, 0]
df_tabula1 = pd.DataFrame()
error_dict = {}

In [23]:
#手順3：tabulaでPDFから特徴量をDataFrameへ記入　※2時間くらいかかる
'''
・取込エラー
    No objects to concatenate　→　データ抽出できないPDFである可能性が高い
'''
for i in range(0, len(codes_tabula)):
    code_tabula = codes_tabula[i]
    start_url_tabula = 'http://ke.kabupro.jp/code/{}.htm'.format(code_tabula)
    print(start_url_tabula, 'i=', i)
    get_pdf_tabula(start_url_tabula, code_tabula)

http://ke.kabupro.jp/code/1333.htm i= 0
http://ke.kabupro.jp/code/1438.htm i= 1
http://ke.kabupro.jp/code/1447.htm i= 2
http://ke.kabupro.jp/code/1448.htm i= 3
http://ke.kabupro.jp/code/1451.htm i= 4
http://ke.kabupro.jp/code/1712.htm i= 5
http://ke.kabupro.jp/code/1721.htm i= 6
http://ke.kabupro.jp/code/1723.htm i= 7
取込エラー
No objects to concatenate


http://ke.kabupro.jp/code/1724.htm i= 8
http://ke.kabupro.jp/code/1726.htm i= 9
http://ke.kabupro.jp/code/1737.htm i= 10
http://ke.kabupro.jp/code/1766.htm i= 11
取込エラー
No objects to concatenate


http://ke.kabupro.jp/code/1780.htm i= 12
取込エラー
No objects to concatenate


http://ke.kabupro.jp/code/1798.htm i= 13
http://ke.kabupro.jp/code/1801.htm i= 14
http://ke.kabupro.jp/code/1803.htm i= 15
http://ke.kabupro.jp/code/1812.htm i= 16
http://ke.kabupro.jp/code/1820.htm i= 17
http://ke.kabupro.jp/code/1824.htm i= 18
http://ke.kabupro.jp/code/1828.htm i= 19
http://ke.kabupro.jp/code/1840.htm i= 20
http://ke.kabupro.jp/code/1847.htm i= 21
http:/

http://ke.kabupro.jp/code/3540.htm i= 177
http://ke.kabupro.jp/code/3541.htm i= 178
http://ke.kabupro.jp/code/3543.htm i= 179
http://ke.kabupro.jp/code/3550.htm i= 180
http://ke.kabupro.jp/code/3556.htm i= 181
http://ke.kabupro.jp/code/3559.htm i= 182
http://ke.kabupro.jp/code/3562.htm i= 183
http://ke.kabupro.jp/code/3566.htm i= 184
http://ke.kabupro.jp/code/3577.htm i= 185
http://ke.kabupro.jp/code/3607.htm i= 186
http://ke.kabupro.jp/code/3626.htm i= 187
http://ke.kabupro.jp/code/3627.htm i= 188
http://ke.kabupro.jp/code/3628.htm i= 189
取込エラー
No objects to concatenate


http://ke.kabupro.jp/code/3630.htm i= 190
http://ke.kabupro.jp/code/3636.htm i= 191
http://ke.kabupro.jp/code/3640.htm i= 192
http://ke.kabupro.jp/code/3646.htm i= 193
http://ke.kabupro.jp/code/3647.htm i= 194
http://ke.kabupro.jp/code/3649.htm i= 195
http://ke.kabupro.jp/code/3652.htm i= 196
取込エラー
No objects to concatenate


http://ke.kabupro.jp/code/3653.htm i= 197
http://ke.kabupro.jp/code/3665.htm i= 198
http://k

取込エラー
No objects to concatenate


http://ke.kabupro.jp/code/4680.htm i= 356
http://ke.kabupro.jp/code/4709.htm i= 357
http://ke.kabupro.jp/code/4714.htm i= 358
http://ke.kabupro.jp/code/4719.htm i= 359
http://ke.kabupro.jp/code/4722.htm i= 360
http://ke.kabupro.jp/code/4732.htm i= 361
http://ke.kabupro.jp/code/4752.htm i= 362
http://ke.kabupro.jp/code/4760.htm i= 363
http://ke.kabupro.jp/code/4761.htm i= 364
取込エラー
No objects to concatenate


http://ke.kabupro.jp/code/4771.htm i= 365
http://ke.kabupro.jp/code/4812.htm i= 366
http://ke.kabupro.jp/code/4819.htm i= 367
取込エラー
No objects to concatenate


http://ke.kabupro.jp/code/4820.htm i= 368
http://ke.kabupro.jp/code/4826.htm i= 369
http://ke.kabupro.jp/code/4838.htm i= 370
http://ke.kabupro.jp/code/4840.htm i= 371
http://ke.kabupro.jp/code/4888.htm i= 372
http://ke.kabupro.jp/code/4901.htm i= 373
http://ke.kabupro.jp/code/4917.htm i= 374
http://ke.kabupro.jp/code/4918.htm i= 375
http://ke.kabupro.jp/code/4922.htm i= 376
http://ke.kabupr

http://ke.kabupro.jp/code/6664.htm i= 530
http://ke.kabupro.jp/code/6696.htm i= 531
http://ke.kabupro.jp/code/6701.htm i= 532
http://ke.kabupro.jp/code/6702.htm i= 533
http://ke.kabupro.jp/code/6703.htm i= 534
http://ke.kabupro.jp/code/6724.htm i= 535
http://ke.kabupro.jp/code/6730.htm i= 536
http://ke.kabupro.jp/code/6737.htm i= 537
http://ke.kabupro.jp/code/6752.htm i= 538
取込エラー
No objects to concatenate


http://ke.kabupro.jp/code/6753.htm i= 539
http://ke.kabupro.jp/code/6754.htm i= 540
http://ke.kabupro.jp/code/6762.htm i= 541
取込エラー
No objects to concatenate


http://ke.kabupro.jp/code/6777.htm i= 542
http://ke.kabupro.jp/code/6794.htm i= 543
http://ke.kabupro.jp/code/6804.htm i= 544
http://ke.kabupro.jp/code/6806.htm i= 545
http://ke.kabupro.jp/code/6809.htm i= 546
取込エラー
No objects to concatenate


http://ke.kabupro.jp/code/6817.htm i= 547
http://ke.kabupro.jp/code/6823.htm i= 548
取込エラー
No objects to concatenate


http://ke.kabupro.jp/code/6824.htm i= 549
http://ke.kabupro.jp/cod

http://ke.kabupro.jp/code/8225.htm i= 709
http://ke.kabupro.jp/code/8227.htm i= 710
http://ke.kabupro.jp/code/8230.htm i= 711
http://ke.kabupro.jp/code/8247.htm i= 712
http://ke.kabupro.jp/code/8255.htm i= 713
http://ke.kabupro.jp/code/8260.htm i= 714
取込エラー
No objects to concatenate


http://ke.kabupro.jp/code/8287.htm i= 715
http://ke.kabupro.jp/code/8308.htm i= 716
http://ke.kabupro.jp/code/8336.htm i= 717
http://ke.kabupro.jp/code/8358.htm i= 718
取込エラー
No objects to concatenate


http://ke.kabupro.jp/code/8366.htm i= 719
取込エラー
No objects to concatenate


http://ke.kabupro.jp/code/8387.htm i= 720
取込エラー
No objects to concatenate


http://ke.kabupro.jp/code/8395.htm i= 721
取込エラー
No objects to concatenate


http://ke.kabupro.jp/code/8439.htm i= 722
http://ke.kabupro.jp/code/8511.htm i= 723
http://ke.kabupro.jp/code/8518.htm i= 724
取込エラー
Plan shapes are not aligned


http://ke.kabupro.jp/code/8522.htm i= 725
http://ke.kabupro.jp/code/8560.htm i= 726
http://ke.kabupro.jp/code/8566.htm i= 

In [24]:
#手順3：tabulaで取得した表の確認
df_tabula1 = df_tabula1.reindex(columns=['code', '流動資産合計', '現金及び預金', '現金預金', '有形固定資産', '有形固定資産合計', 
                         '投資有価証券', '投資その他の資産合計', '流動負債合計', '固定負債合計', ])
df_tabula1

,code,流動資産合計,現金及び預金,現金預金,有形固定資産,有形固定資産合計,投資有価証券,投資その他の資産合計,流動負債合計,固定負債合計
0,1333,"300,511","31,579",NaN,NaN,"147,902","43,665","64,393","213,484","152,237"
0,1438,"3,239,976","2,125,532",NaN,NaN,"971,599",NaN,"312,646","1,278,725","232,571"
0,1447,"10,852,865","3,894,837",NaN,NaN,"1,720,706","215,825","1,246,302","7,571,502","4,709,414"
0,1448,"39,809",NaN,"6,654",NaN,"21,904","11,164","13,113","37,814","15,465"
0,1451,"12,489,804","3,984,906",NaN,NaN,"1,946,262","1,689","282,510","5,000,041","4,076,325"
...,...,...,...,...,...,...,...,...,...,...
0,9964,"38,009,140","6,437,893",NaN,NaN,"21,001,210","665,558","2,250,509","25,691,945","4,503,249"
0,9978,"9,068,441","2,105,930",NaN,NaN,"917,594","92,427","1,480,395","7,690,898","2,883,576"
0,9983,"1,729,266",NaN,NaN,"151,875",NaN,NaN,NaN,"631,623",NaN
0,9984,"△4,816,777",NaN,NaN,"404,062 H",NaN,"2,495,273 M",NaN,NaN,NaN


In [25]:
#手順3：tabulaで取得した表をcsvへ書込み
df_tabula1.to_csv('temp_file/全銘柄PDFデータ（手順3_tabula加工前）.csv', mode='w',
                  index=False, encoding="shift-jis", errors='none')

In [26]:
#手順3：tabulaで取得した表をcsvから読み込み、余分な文字を消す
df_tabula2 = pd.read_csv('temp_file/全銘柄PDFデータ（手順3_tabula加工前）.csv', encoding=("cp932"), header=0)

replace_feature_list = list(df_tabula2.select_dtypes(include=object).columns.values)
replace_list = [',', '△', '―', '-', '，', '\n', ' ',
                '※1', '※2', '※3', '※4', '※5', '※6', '※7', '※8', '※9',
                '※１', '※２', '※３', '※４', '※５', '※６', '※７', '※８', '※９', '\D',
                'H', 'M', '百万円']
for i in replace_feature_list:
    for j in replace_list:
        df_tabula2[i] = df_tabula2[i].str.replace(j, '')
df_tabula2

,code,流動資産合計,現金及び預金,現金預金,有形固定資産,有形固定資産合計,投資有価証券,投資その他の資産合計,流動負債合計,固定負債合計
0,1333,300511,31579,NaN,NaN,147902,43665,64393,213484,152237
1,1438,3239976,2125532,NaN,NaN,971599,NaN,312646,1278725,232571
2,1447,10852865,3894837,NaN,NaN,1720706,215825,1246302,7571502,4709414
3,1448,39809,NaN,6654,NaN,21904,11164,13113,37814,15465
4,1451,12489804,3984906,NaN,NaN,1946262,1689,282510,5000041,4076325
...,...,...,...,...,...,...,...,...,...,...
713,9964,38009140,6437893,NaN,NaN,21001210,665558,2250509,25691945,4503249
714,9978,9068441,2105930,NaN,NaN,917594,92427,1480395,7690898,2883576
715,9983,1729266,NaN,NaN,151875,NaN,NaN,NaN,631623,NaN
716,9984,4816777,NaN,NaN,404062,NaN,2495273,NaN,NaN,NaN


In [27]:
#手順3：tabulaで再取得した銘柄のうち、十分に取り込めていない銘柄コードの抽出と削除
#tabulaはあまりうまく抽出できないため、削除する銘柄が多い
tabula_drop_list = []

#削除する銘柄コードの抽出        
for i in range(0, len(df_tabula2)):    
    if len(df_tabula2.iloc[i, :].value_counts()) <= 3:
        tabula_drop_list.append(df_tabula2.iloc[i, 0])
        print('code=', df_tabula2.iloc[i, 0], '削除：特徴量が2つ以下')
        
#削除する銘柄コードの削除
for i in tabula_drop_list:
    df_tabula2.drop(df_tabula2.index[df_tabula2['code'] == i], inplace=True)
df_tabula2

code= 2160 削除：特徴量が2つ以下
code= 2175 削除：特徴量が2つ以下
code= 2914 削除：特徴量が2つ以下
code= 2928 削除：特徴量が2つ以下
code= 3035 削除：特徴量が2つ以下
code= 3673 削除：特徴量が2つ以下
code= 3837 削除：特徴量が2つ以下
code= 3996 削除：特徴量が2つ以下
code= 4021 削除：特徴量が2つ以下
code= 4061 削除：特徴量が2つ以下
code= 4151 削除：特徴量が2つ以下
code= 4235 削除：特徴量が2つ以下
code= 4503 削除：特徴量が2つ以下
code= 4541 削除：特徴量が2つ以下
code= 4578 削除：特徴量が2つ以下
code= 4612 削除：特徴量が2つ以下
code= 4901 削除：特徴量が2つ以下
code= 4985 削除：特徴量が2つ以下
code= 5108 削除：特徴量が2つ以下
code= 5189 削除：特徴量が2つ以下
code= 5192 削除：特徴量が2つ以下
code= 5357 削除：特徴量が2つ以下
code= 5857 削除：特徴量が2つ以下
code= 5922 削除：特徴量が2つ以下
code= 5949 削除：特徴量が2つ以下
code= 6034 削除：特徴量が2つ以下
code= 6287 削除：特徴量が2つ以下
code= 6301 削除：特徴量が2つ以下
code= 6324 削除：特徴量が2つ以下
code= 6376 削除：特徴量が2つ以下
code= 6406 削除：特徴量が2つ以下
code= 6448 削除：特徴量が2つ以下
code= 6481 削除：特徴量が2つ以下
code= 6506 削除：特徴量が2つ以下
code= 6702 削除：特徴量が2つ以下
code= 6724 削除：特徴量が2つ以下
code= 6754 削除：特徴量が2つ以下
code= 6869 削除：特徴量が2つ以下
code= 6942 削除：特徴量が2つ以下
code= 6988 削除：特徴量が2つ以下
code= 7013 削除：特徴量が2つ以下
code= 7122 削除：特徴量が2つ以下
code= 7198 削除：特徴量が2つ以下
code= 7282 

,code,流動資産合計,現金及び預金,現金預金,有形固定資産,有形固定資産合計,投資有価証券,投資その他の資産合計,流動負債合計,固定負債合計
0,1333,300511,31579,NaN,NaN,147902,43665,64393,213484,152237
1,1438,3239976,2125532,NaN,NaN,971599,NaN,312646,1278725,232571
2,1447,10852865,3894837,NaN,NaN,1720706,215825,1246302,7571502,4709414
3,1448,39809,NaN,6654,NaN,21904,11164,13113,37814,15465
4,1451,12489804,3984906,NaN,NaN,1946262,1689,282510,5000041,4076325
...,...,...,...,...,...,...,...,...,...,...
713,9964,38009140,6437893,NaN,NaN,21001210,665558,2250509,25691945,4503249
714,9978,9068441,2105930,NaN,NaN,917594,92427,1480395,7690898,2883576
715,9983,1729266,NaN,NaN,151875,NaN,NaN,NaN,631623,NaN
716,9984,4816777,NaN,NaN,404062,NaN,2495273,NaN,NaN,NaN


In [28]:
#手順3：tabulaで取得した表（加工・削除後）をcsvへ書込み
df_tabula2.to_csv('temp_file/全銘柄PDFデータ（手順3_tabula完成）.csv', mode='w',
                      index=False, encoding="shift-jis", errors='none')

In [32]:
#手順4：camelotとcamelot_reとtabulaの読出し→結合
df_camelot = pd.read_csv('temp_file/全銘柄PDFデータ（手順1_camelot完成）.csv', 
                             encoding=("cp932"), header=0)
df_camelot_re = pd.read_csv('temp_file/全銘柄PDFデータ（手順2_camelot_re完成）.csv', 
                             encoding=("cp932"), header=0)
df_tabula = pd.read_csv('temp_file/全銘柄PDFデータ（手順3_tabula完成）.csv', 
                             encoding=("cp932"), header=0)

df_pdf = pd.concat([df_camelot, df_camelot_re, df_tabula], axis=0)
df_pdf

,code,流動資産合計,現金及び預金,現金預金,有形固定資産,有形固定資産合計,投資有価証券,投資その他の資産合計,流動負債合計,固定負債合計
0,1301,86649,7100,NaN,NaN,15715.0,11682.0,13547.0,39189,37166.0
1,1332,231751,10986,NaN,NaN,147428.0,38076.0,86443.0,153467,134021.0
2,1352,12137,4019,NaN,NaN,26593.0,345.0,2390.0,11866,21666.0
3,1375,10202,NaN,NaN,18031.0,NaN,NaN,NaN,7614,NaN
4,1376,28828145,3447948,NaN,NaN,6627416.0,NaN,2829597.0,16514107,1726482.0
...,...,...,...,...,...,...,...,...,...,...
646,9964,3.80091e+07,6.43789e+06,NaN,NaN,21001210.0,665558.0,2250509.0,2.56919e+07,4503249.0
647,9978,9.06844e+06,2.10593e+06,NaN,NaN,917594.0,92427.0,1480395.0,7.6909e+06,2883576.0
648,9983,1.72927e+06,NaN,NaN,151875.0,NaN,NaN,NaN,631623,NaN
649,9984,4.81678e+06,NaN,NaN,404062.0,NaN,2495273.0,NaN,NaN,NaN


In [33]:
#手順4：「現金預金」の値を「現金及び預金」に統合
for i in range(0, len(df_pdf)):
    if math.isnan(df_pdf.iloc[i, 3]) is False:
        df_pdf.iloc[i, 2] = df_pdf.iloc[i, 3]
del df_pdf['現金預金']

df_pdf

,code,流動資産合計,現金及び預金,有形固定資産,有形固定資産合計,投資有価証券,投資その他の資産合計,流動負債合計,固定負債合計
0,1301,86649,7100,NaN,15715.0,11682.0,13547.0,39189,37166.0
1,1332,231751,10986,NaN,147428.0,38076.0,86443.0,153467,134021.0
2,1352,12137,4019,NaN,26593.0,345.0,2390.0,11866,21666.0
3,1375,10202,NaN,18031.0,NaN,NaN,NaN,7614,NaN
4,1376,28828145,3447948,NaN,6627416.0,NaN,2829597.0,16514107,1726482.0
...,...,...,...,...,...,...,...,...,...
646,9964,3.80091e+07,6.43789e+06,NaN,21001210.0,665558.0,2250509.0,2.56919e+07,4503249.0
647,9978,9.06844e+06,2.10593e+06,NaN,917594.0,92427.0,1480395.0,7.6909e+06,2883576.0
648,9983,1.72927e+06,NaN,151875.0,NaN,NaN,NaN,631623,NaN
649,9984,4.81678e+06,NaN,404062.0,NaN,2495273.0,NaN,NaN,NaN


In [43]:
#手順4：結合した表をcsvへ書込み
df_pdf.to_csv('全銘柄PDFデータ（完成）.csv', mode='w',
                      index=False, encoding="shift-jis", errors='none')
df_pdf.dtypes

code            int64
流動資産合計         object
現金及び預金         object
有形固定資産        float64
有形固定資産合計      float64
投資有価証券        float64
投資その他の資産合計    float64
流動負債合計         object
固定負債合計        float64
dtype: object

In [46]:
#手順5：データのMySQLへの取込み
import mysql.connector
import csv
config = {'user':'root',
    'password' : '',
    'host' : '127.0.0.1',
    'database' : 'test',
    'use_unicode' : 'True',
    'charset' :'utf8' }
dbconnector = mysql.connector.connect(**config) 
# 接続できているか確認 
if dbconnector.is_connected():
    print("データベースへの接続が成功しました。") 
else:
    print("データベースへの接続が失敗しました。") 
    exit(1) 
# cursor オブジェクトの作成 
cursor = dbconnector.cursor(buffered = True) 

cursor.execute("""
    CREATE TABLE IF NOT EXISTS all_stock_PDFdata (
        `code` INTEGER NOT NULL ,
        `流動資産合計` INTEGER,
        `現金及び預金` INTEGER,
        `有形固定資産` INTEGER,
        `有形固定資産合計` INTEGER,
        `投資有価証券` INTEGER,
        `投資その他の資産合計` INTEGER,
        `流動負債合計` INTEGER,
        `固定負債合計` INTEGER,
        PRIMARY KEY (`code`)
        
    )
""")

operation = ("""INSERT INTO all_stock_PDFdata
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)""")
seqs = [] 
with open("全銘柄PDFデータ（完成）.csv", encoding='shift-jis') as f:
    reader = csv.reader(f)
    for row in reader:
        tuples=(row[0], row[1], row[2], row[3], row[4],
                row[5], row[6], row[7], row[8],
               )
        seqs.append(tuples,)
for i in range(1, len(seqs)):
    cursor.execute(operation, seqs[i]) 

dbconnector.commit()
dbconnector.close()

データベースへの接続が成功しました。


In [ ]:
###【参考】

#camelotでPDFから特徴量を「並列処理で」DataFrameへ記入　※通常の80%の時間で取り込むことが出来る
import os
import concurrent.futures as confu

with confu.ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
    for i in range(0, 30):
        code = codes[i]
        start_url = 'http://ke.kabupro.jp/code/{}.htm'.format(code)
        time.sleep(1)
        print(start_url, 'i=', i)
        executor.submit(get_pdf_camelot, start_url, code, i)